# ViralCutter
Uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`

# Suporte em:
[![](https://dcbadge.limes.pink/api/server/tAdPHFAbud)](https://discord.gg/tAdPHFAbud)

# TODO📝
- [x] Release code
- [ ] Huggingface SpaceDemo
- [x] Two face in the cut
- [x] Custom caption and burn
- [ ] Make the code faster
- [ ] More types of framing beyond 9:16

In [ ]:
#@title 🛠️ Instalação
import os
import subprocess
import shutil
from IPython.display import clear_output

# 1. Limpeza TOTAL
print("🧹 Limpando instalação anterior...")
%cd /content
if os.path.exists("ViralCutter"):
    shutil.rmtree("ViralCutter")

!git clone -b dev https://github.com/RafaelGodoyEbert/ViralCutter.git
%cd /content/ViralCutter

print("⏳ Instalando gerenciador UV e drivers do sistema...")

# 2. Instalar UV e Drivers Linux
subprocess.run(['pip', 'install', 'uv'], check=True)
subprocess.run('sudo apt update -y && sudo apt install -y libcudnn8 ffmpeg xvfb', shell=True, check=True)

# 3. Criar Ambiente Virtual
print("⏳ Criando ambiente virtual...")
subprocess.run(['uv', 'venv', '.venv'], check=True)

# 4. INSTALAÇÃO DAS DEPENDÊNCIAS
print("⏳ Instalando Bibliotecas...")

# Passo A: WhisperX e Requisitos Básicos (Deixe instalar o que quiserem)
cmds_fase_1 = [
    "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
    "uv pip install --python .venv -r requirements.txt",
    "uv pip install --python .venv yt-dlp pytubefix"
]

for cmd in cmds_fase_1:
    subprocess.run(cmd, shell=True, check=True)

# Passo B: CORREÇÃO DE ALINHAMENTO E GEMINI
# - google-generativeai: Para o Gemini funcionar
# - pandas: Para separar palavras
# - transformers==4.46.3: VERSÃO CRÍTICA. Versões mais novas exigem Torch 2.6 e quebram o alinhamento.
# - accelerate: Ajuda no carregamento do modelo
print("🔨 Aplicando downgrade estratégico no Transformers...")
extra_libs = [
    "uv pip install --python .venv google-generativeai",
    "uv pip install --python .venv pandas",
    "uv pip install --python .venv onnxruntime-gpu",
    "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0"
]

for cmd in extra_libs:
    subprocess.run(cmd, shell=True, check=True)

# Passo C: O MARTELO FINAL (Torch 2.3.1 Estável)
# Reinstalamos por último para garantir que nada atualizou ele sem querer
print("🔨 Forçando versão estável do Torch (2.3.1)...")
cmd_fix_torch = (
    "uv pip install --python .venv "
    "torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 "
    "--index-url https://download.pytorch.org/whl/cu121"
)
subprocess.run(cmd_fix_torch, shell=True, check=True)

# Passo D: Trava do Numpy
print("🔨 Travando Numpy...")
subprocess.run("uv pip install --python .venv 'numpy<2.0' setuptools==69.5.1", shell=True, check=True)

# 5. Configurar Monitor
os.system('Xvfb :1 -screen 0 2560x1440x8 &')
os.environ['DISPLAY'] = ':1.0'

clear_output()
print("✅ Instalação V7 Finalizada!")
print("- Transformers 4.46.3 (Compatível com Alinhamento): INSTALADO")
print("- Torch 2.3.1: ATIVO")

In [ ]:
#@title 🚀 Configuração e Execução
%cd /content/ViralCutter
import os
import json
import subprocess

# --- FIX MATPLOTLIB ---
os.environ['MPLBACKEND'] = 'Agg'

# --- VERIFICAÇÃO DE PRÉ-VOO ---
venv_python = "/content/ViralCutter/.venv/bin/python"
print("🔍 Verificando bibliotecas críticas...")
try:
    subprocess.check_output([venv_python, "-c", "import google.generativeai; print('Gemini OK')"])
    print("✅ Gemini (google-generativeai): OK")
except:
    print("❌ ERRO: google-generativeai não encontrado!")

try:
    subprocess.check_output([venv_python, "-c", "import pandas; print('Pandas OK')"])
    print("✅ Pandas (Alinhamento): OK")
except:
    print("❌ ERRO: Pandas não encontrado (Alinhamento vai falhar)!")

#@markdown # 🚀 **Configuração Geral**
url = 'https://youtu.be/1ZnwqFh3CrA?si=zpSig52k11NY7que'  # @param {type: "string"}

#@markdown ---
#@markdown ### 🤖 **Inteligência Artificial**
ai_backend = "gemini" #@param ["gemini", "g4f", "manual"]
api_key = "" #@param {type:"string"}

#@markdown ---
#@markdown ### 🎬 **Configurações do Vídeo**
num_segments = 3  # @param {type:"number"}
viral_mode = True #@param{type:"boolean"}
themes = '' # @param {type:"string", placeholder:"Se `viral_mode` desmarcado, coloque o tema aqui"}
model = 'large-v3-turbo' # @param ['base', 'large-v3', 'large-v3-turbo']

#@markdown ---
#@markdown ### ⏳ **Configurações de Tempo**
tempo_minimo = 15 #@param{type: "number"}
tempo_maximo = 90 #@param{type: "number"}

#@markdown ---
#@markdown ### ⚙️ **Configurações Avançadas de Legenda**
advanced_settings = False #@param {type:"boolean"}
burn_subtitles_option = True # @param {type:"boolean"}

#@markdown *(As opções abaixo só funcionam se "advanced_settings" estiver ativado)*

#@markdown ### Fonte da legenda
fonte = "Arial" #@param{type: "string"}
#@markdown ### Tamanho da fonte
base_size = 12 #@param{type: "number"}
#@markdown ### Cor da fonte
base_color_c = "FFFFFF" #@param{type: "string"}
#@markdown ### Transparência da fonte (00=opaco, FF=transparente)
base_color_t = "00" #@param{type: "string"}

#@markdown ### Cor do contorno
contorno_c = "808080" #@param{type: "string"}
#@markdown ### Transparência do contorno
contorno_t = "00" #@param{type: "string"}

#@markdown ### Tamanho do highlight
h_size = 14 #@param{type: "number"}
#@markdown ### Cor do destaque
highlight_color_c = "00FF00" #@param{type: "string"}

#@markdown ### Palavras por bloco
palavras_por_bloco = 5 #@param{type: "number"}
#@markdown ### Limite de gap (segundos)
limite_gap = 0.5 #@param{type: "number"}
#@markdown ### Modo de exibição
modo = 'highlight' # @param ['highlight', 'sem_higlight', 'palavra_por_palavra']
#@markdown ### Posição vertical (default 60)
posicao_vertical = 60 #@param{type: "number"}
#@markdown ### Alinhamento (2=Centro)
alinhamento = 2 #@param{type: "slider", min: 1, max: 3}

#@markdown ### Estilo de borda (1=Contorno, 3=Caixa)
estilo_da_borda = 3 #@param{type: "number"}
#@markdown ### Espessura do contorno
espessura_do_contorno = 1.5 #@param{type: "number"}
#@markdown ### Tamanho da sombra
tamanho_da_sombra = 10 #@param{type: "number"}

# Outras configs fixas para avançado
negrito = 0
italico = 0
sublinhado = 0
tachado = 0
cor_da_sombra_c = "000000"
cor_da_sombra_t = "00"
highlight_color_t = "00"

burn_only = False #@param {type:"boolean"}

# Create config file if advanced settings are enabled
config_json_path = "subtitle_config.json"
use_subtitle_config = False

if advanced_settings:
    def make_color(color_hex, transparency_hex):
        return f"&H{transparency_hex}" + color_hex + "&"

    config_data = {
        "font": fonte,
        "base_size": base_size,
        "base_color": make_color(base_color_c, base_color_t),
        "highlight_size": h_size,
        "highlight_color": make_color(highlight_color_c, "00"),
        "words_per_block": palavras_por_bloco,
        "gap_limit": limite_gap,
        "mode": modo,
        "vertical_position": posicao_vertical,
        "alignment": alinhamento,
        "border_style": estilo_da_borda,
        "outline_thickness": espessura_do_contorno,
        "outline_color": make_color(contorno_c, contorno_t),
        "shadow_size": tamanho_da_sombra,
        "shadow_color": make_color(cor_da_sombra_c, cor_da_sombra_t),
        "bold": negrito,
        "italic": italico,
        "underline": sublinhado,
        "strikeout": tachado
    }

    with open(config_json_path, "w", encoding="utf-8") as f:
        json.dump(config_data, f, indent=4)
    use_subtitle_config = True
    print("Advanced configuration saved.")

# --- COMANDO ---
cmd = f"{venv_python} main_improved.py --url \"{url}\" --segments {num_segments}"

if viral_mode:
    cmd += " --viral"
else:
    if themes:
        cmd += f" --themes \"{themes}\""

cmd += f" --min-duration {tempo_minimo} --max-duration {tempo_maximo} --model {model}"
cmd += f" --ai-backend {ai_backend}"

if api_key:
    cmd += f" --api-key \"{api_key}\""

if burn_only:
    cmd += " --burn-only"

cmd += " --workflow 1"
cmd += " --face-model insightface"
cmd += " --face-mode auto"
cmd += " --skip-prompts"

if use_subtitle_config:
    cmd += f" --subtitle-config {config_json_path}"

print(f"Executing: {cmd}")
print("Starting ViralCutter Pipeline...")

!{cmd}

print("Process finished!")

# Resultado em vídeo

In [ ]:
#@title Resultado em vídeo
#@markdown <h1><b> Escolha o número do vídeo e o nome do projeto
from IPython.display import HTML
from base64 import b64encode
import json
import os

# --- PARÂMETROS ---
nome_do_projeto = "Unknown_Video" #@param {type:"string"}
numero_do_video = 0 #@param {type:"number"}
tipo_de_video = "Legendas queimadas" # @param ["Legendas queimadas", "Video limpo"]

# --- DEFINIÇÃO DOS CAMINHOS ---
# Define a pasta raiz do projeto ex: /content/ViralCutter/VIRALS/Unknown_Video
base_dir = f'/content/ViralCutter/VIRALS/{nome_do_projeto}'

# Define o caminho exato do arquivo de segmentos dentro da pasta do projeto
segments_path = f'{base_dir}/viral_segments.txt'

if tipo_de_video == "Legendas queimadas":
  pasta_video = 'burned_sub'
  sufix = 'processed_subtitled'
else:
  pasta_video = 'final'
  sufix = 'processed'

# --- FUNÇÕES ---

def show_video(video_path, video_width=400):
    if not os.path.exists(video_path):
        return HTML(f"<h3 style='color:red'>Erro: Vídeo não encontrado!</h3><p>Caminho tentado: {video_path}</p>")

    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

# Carregar informações do arquivo de texto (JSON)
segments_data = None
if os.path.exists(segments_path):
    try:
        with open(segments_path, 'r') as file:
            segments_data = json.load(file)
    except Exception as e:
        print(f"Erro ao ler o arquivo JSON: {e}")
else:
    print(f"ALERTA: Arquivo '{segments_path}' não encontrado. As informações de texto não serão exibidas.")

# Função para exibir o vídeo e as informações
def display_video_with_segment(index):
    # Exibir textos se o JSON foi carregado
    if segments_data:
        try:
            segment = segments_data['segments'][index]
            print(f"Título: {segment.get('title', 'Sem título')}")
            print(f"Descrição: {segment.get('description', 'Sem descrição')}")
            print(f"Score: {segment.get('score', 'N/A')}")
            print("-" * 30)
        except IndexError:
            print(f"Aviso: Não há informações de texto para o índice {index}.")
        except KeyError:
            print("Aviso: Formato do JSON incorreto.")

    # Caminho do vídeo
    # Ex: /content/ViralCutter/VIRALS/Unknown_Video/burned_sub/final-output000_processed_subtitled.mp4
    nome_arquivo = f'final-output{str(index).zfill(3)}_{sufix}.mp4'
    caminho_completo_video = f'{base_dir}/{pasta_video}/{nome_arquivo}'

    return show_video(caminho_completo_video)

# Executar
display_video_with_segment(numero_do_video)

## Baixar Zip | Off

In [ ]:
#@title Baixar <b><u>todos</u> os virais gerados
# prompt: Zipar e baixar todos os arquivos da pasta /content/final

!zip -r /content/final.zip /content/ViralCutter/final
from google.colab import files
files.download("/content/final.zip")


In [ ]:
#@title Baixar somente Cortes com <u>resolução original 💻
#@markdown Logo vai baixar os vídeos que são somente os cortes, sem a edição e foco no rosto
import os
import zipfile

# Definir o diretório onde os vídeos estão localizados
directory = '/content/ViralCutter/tmp'
zip_filename = '/content/ViralCutter/videos_original_scale.zip'

# Criar um arquivo zip
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Iterar sobre os arquivos no diretório
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('_original_scale.mp4'):
                # Adicionar o arquivo ao zip
                zipf.write(os.path.join(root, file), arcname=file)

# Fazer o download do arquivo zip
from google.colab import files
files.download(zip_filename)


In [ ]:
#@title Baixar todos os virais gerados <b><u>(com legendas queimadas)

!zip -r /content/burned_sub.zip /content/ViralCutter/burned_sub
from google.colab import files
files.download("/content/burned_sub.zip")


#Créditos

Inspirado no [reels clips automator](https://github.com/eddieoz/reels-clips-automator) e no [YoutubeVideoToAIPoweredShorts](https://github.com/Fitsbit/YoutubeVideoToAIPoweredShorts)<br>

---
![Rafa.png](https://i.imgur.com/cGknQpU.png;base64)

Desenvolvido por **Rafa.Godoy**<br>
[ ![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white) ](https://github.com/rafaelGodoyEbert)<br>
[ ![X](https://img.shields.io/twitter/url?url=https%3A%2F%2Ftwitter.com%2FGodoyEbert) ](https://twitter.com/GodoyEbert)<br>
[Instagram](https://www.instagram.com/rafael.godoy.ebert/)<br>
[ ![](https://dcbadge.vercel.app/api/server/aihubbrasil) ](https://discord.gg/aihubbrasil)

`0.7v Alpha`<br>

Apenas uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`<br>
